In [5]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import datetime
from scipy import stats
from scipy.stats import linregress
from config import weather_api_key
from config import g_key
from citipy import citipy

In [6]:
#set "cit center" for Houston
Houston_lat = 29.756097853207656
Houston_lng = -95.3669907972107


#calculate greater Houston area by expanding lat and lng
lat = np.random.uniform(Houston_lat-1, Houston_lat+2, size=6500)
lng = np.random.uniform(Houston_lng-5, Houston_lng+2, size=6500)
city = []

#find nearest city using lat and lng
for x in range(len(lat)):
    city.append(citipy.nearest_city(lat[x], lng[x]).city_name)


In [14]:
#turn cities into df
df = pd.DataFrame({
    "City":city,
    "lat":lat,
    "lng":lng,
})

In [15]:
df.head()

,City,lat,lng
0,bellmead,31.712121,-96.904123
1,port arthur,29.718811,-93.995816
2,uvalde,28.911333,-99.542573
3,victoria,28.875855,-97.278976
4,nacogdoches,31.533730,-94.586603


In [16]:
#check length of cities
len(list(df["City"].unique()))

114

In [90]:
cities = list(df["City"].unique())
target_cities=[]

for i in cities:
    c=str(i).replace(" ","+")
    target_cities.append(c)

target_cities

['bellmead',
 'port+arthur',
 'uvalde',
 'victoria',
 'nacogdoches',
 'fredericksburg',
 'kirby',
 'beaumont',
 'hondo',
 'angleton',
 'canyon+lake',
 'beeville',
 'lockhart',
 'kerrville',
 'lumberton',
 'palestine',
 'missouri+city',
 'brownwood',
 'galveston',
 'wharton',
 'sulphur',
 'gatesville',
 'el+campo',
 'natchitoches',
 'cloverleaf',
 'brenham',
 'pearland',
 'temple',
 'pleasanton',
 'del+rio',
 'taylor',
 'baytown',
 'georgetown',
 'woodway',
 'pflugerville',
 'bryan',
 'katy',
 'lufkin',
 'rosenberg',
 'orange',
 'huntsville',
 'corsicana',
 'college+station',
 'copperas+cove',
 'schertz',
 'freeport',
 'leon+valley',
 'atascocita',
 'conroe',
 'converse',
 'san+marcos',
 'seguin',
 'san+antonio',
 'nederland',
 'spring',
 'alvin',
 'lakeway',
 'austin',
 'san+angelo',
 'lake+jackson',
 'la+marque',
 'liberty',
 'bay+city',
 'eagle+pass',
 'leander',
 'vidor',
 'hewitt',
 'la+porte',
 'the+woodlands',
 'tomball',
 'friendswood',
 'robinson',
 'sugar+land',
 'belton',
 'h

In [79]:
#set function to get today and previous 4 days' weather data
def unix_time():
    today = datetime.date.today()
    days = [1,2,3,4]
    times = [int(time.mktime(today.timetuple()))]
    for day in days:
        x=today - datetime.timedelta(days=day)
        unixtime = int(time.mktime(x.timetuple()))
        times.append(unixtime)
    return (times)

In [18]:
unix_time()

[1630213200, 1630126800, 1630040400, 1629954000, 1629867600]

In [89]:
query_url=f"{base_current_url}appid={weather_api_key}&q=Port+Arthur,US-TX,US&units=imperial"
print(query_url)

http://api.openweathermap.org/data/2.5/weather?appid=2619cb1052a25e553478de5c60d2a13f&q=Port+Arthur,US-TX,US&units=imperial


In [91]:
#set base url and define function to find OpenWeather cities' based on city name from nearest place API and feed back into
#OpenWeather one call API to ensure proper lat and lng


def find_location(target_cities):
    unix_time()
    base_current_url = "http://api.openweathermap.org/data/2.5/weather?"
    for city in target_cities:
        try: 
            query_url=f"{base_current_url}appid={weather_api_key}&q={city},US-TX,US&units=imperial"
            req = requests.get(query_url)
            response1=req.json()
            latitude=response1["coord"]["lat"]
            longitude=response1["coord"]["lon"]
        except:
            print("City not found. Skipping...")
            break
        for x in unix_time():
            historical_url = f"http://api.openweathermap.org/data/2.5/onecall/timemachine?lat={latitude}&lon={longitude}&dt={x}&appid={weather_api_key}"
            print(historical_url)
        print ("--"*30)

In [92]:
find_location(target_cities)

http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=31.5941&lon=-97.1089&dt=1630213200&appid=2619cb1052a25e553478de5c60d2a13f
http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=31.5941&lon=-97.1089&dt=1630126800&appid=2619cb1052a25e553478de5c60d2a13f
http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=31.5941&lon=-97.1089&dt=1630040400&appid=2619cb1052a25e553478de5c60d2a13f
http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=31.5941&lon=-97.1089&dt=1629954000&appid=2619cb1052a25e553478de5c60d2a13f
http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=31.5941&lon=-97.1089&dt=1629867600&appid=2619cb1052a25e553478de5c60d2a13f
------------------------------------------------------------
http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=29.8988&lon=-93.9288&dt=1630213200&appid=2619cb1052a25e553478de5c60d2a13f
http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=29.8988&lon=-93.9288&dt=1630126800&appid=2619cb1052a25

http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=28.4008&lon=-97.7483&dt=1630213200&appid=2619cb1052a25e553478de5c60d2a13f
http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=28.4008&lon=-97.7483&dt=1630126800&appid=2619cb1052a25e553478de5c60d2a13f
http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=28.4008&lon=-97.7483&dt=1630040400&appid=2619cb1052a25e553478de5c60d2a13f
http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=28.4008&lon=-97.7483&dt=1629954000&appid=2619cb1052a25e553478de5c60d2a13f
http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=28.4008&lon=-97.7483&dt=1629867600&appid=2619cb1052a25e553478de5c60d2a13f
------------------------------------------------------------
http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=-35.2231&lon=146.7171&dt=1630213200&appid=2619cb1052a25e553478de5c60d2a13f
http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=-35.2231&lon=146.7171&dt=1630126800&appid=2619cb1052a

http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=31.7607&lon=-93.086&dt=1630213200&appid=2619cb1052a25e553478de5c60d2a13f
http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=31.7607&lon=-93.086&dt=1630126800&appid=2619cb1052a25e553478de5c60d2a13f
http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=31.7607&lon=-93.086&dt=1630040400&appid=2619cb1052a25e553478de5c60d2a13f
http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=31.7607&lon=-93.086&dt=1629954000&appid=2619cb1052a25e553478de5c60d2a13f
http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=31.7607&lon=-93.086&dt=1629867600&appid=2619cb1052a25e553478de5c60d2a13f
------------------------------------------------------------
http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=49.9889&lon=-96.5667&dt=1630213200&appid=2619cb1052a25e553478de5c60d2a13f
http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=49.9889&lon=-96.5667&dt=1630126800&appid=2619cb1052a25e5534

http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=30.6744&lon=-96.37&dt=1630213200&appid=2619cb1052a25e553478de5c60d2a13f
http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=30.6744&lon=-96.37&dt=1630126800&appid=2619cb1052a25e553478de5c60d2a13f
http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=30.6744&lon=-96.37&dt=1630040400&appid=2619cb1052a25e553478de5c60d2a13f
http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=30.6744&lon=-96.37&dt=1629954000&appid=2619cb1052a25e553478de5c60d2a13f
http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=30.6744&lon=-96.37&dt=1629867600&appid=2619cb1052a25e553478de5c60d2a13f
------------------------------------------------------------
http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=29.7858&lon=-95.8244&dt=1630213200&appid=2619cb1052a25e553478de5c60d2a13f
http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=29.7858&lon=-95.8244&dt=1630126800&appid=2619cb1052a25e553478de5

http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=29.4952&lon=-98.6186&dt=1630213200&appid=2619cb1052a25e553478de5c60d2a13f
http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=29.4952&lon=-98.6186&dt=1630126800&appid=2619cb1052a25e553478de5c60d2a13f
http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=29.4952&lon=-98.6186&dt=1630040400&appid=2619cb1052a25e553478de5c60d2a13f
http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=29.4952&lon=-98.6186&dt=1629954000&appid=2619cb1052a25e553478de5c60d2a13f
http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=29.4952&lon=-98.6186&dt=1629867600&appid=2619cb1052a25e553478de5c60d2a13f
------------------------------------------------------------
http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=29.999&lon=-95.162&dt=1630213200&appid=2619cb1052a25e553478de5c60d2a13f
http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=29.999&lon=-95.162&dt=1630126800&appid=2619cb1052a25e553

http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=31.4648&lon=-100.4398&dt=1630213200&appid=2619cb1052a25e553478de5c60d2a13f
http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=31.4648&lon=-100.4398&dt=1630126800&appid=2619cb1052a25e553478de5c60d2a13f
http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=31.4648&lon=-100.4398&dt=1630040400&appid=2619cb1052a25e553478de5c60d2a13f
http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=31.4648&lon=-100.4398&dt=1629954000&appid=2619cb1052a25e553478de5c60d2a13f
http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=31.4648&lon=-100.4398&dt=1629867600&appid=2619cb1052a25e553478de5c60d2a13f
------------------------------------------------------------
http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=29.0339&lon=-95.4344&dt=1630213200&appid=2619cb1052a25e553478de5c60d2a13f
http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=29.0339&lon=-95.4344&dt=1630126800&appid=2619cb10

http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=30.0972&lon=-95.6161&dt=1630213200&appid=2619cb1052a25e553478de5c60d2a13f
http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=30.0972&lon=-95.6161&dt=1630126800&appid=2619cb1052a25e553478de5c60d2a13f
http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=30.0972&lon=-95.6161&dt=1630040400&appid=2619cb1052a25e553478de5c60d2a13f
http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=30.0972&lon=-95.6161&dt=1629954000&appid=2619cb1052a25e553478de5c60d2a13f
http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=30.0972&lon=-95.6161&dt=1629867600&appid=2619cb1052a25e553478de5c60d2a13f
------------------------------------------------------------
http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=29.5294&lon=-95.201&dt=1630213200&appid=2619cb1052a25e553478de5c60d2a13f
http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=29.5294&lon=-95.201&dt=1630126800&appid=2619cb1052a25e5

http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=28.615&lon=-96.6261&dt=1630213200&appid=2619cb1052a25e553478de5c60d2a13f
http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=28.615&lon=-96.6261&dt=1630126800&appid=2619cb1052a25e553478de5c60d2a13f
http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=28.615&lon=-96.6261&dt=1630040400&appid=2619cb1052a25e553478de5c60d2a13f
http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=28.615&lon=-96.6261&dt=1629954000&appid=2619cb1052a25e553478de5c60d2a13f
http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=28.615&lon=-96.6261&dt=1629867600&appid=2619cb1052a25e553478de5c60d2a13f
------------------------------------------------------------
http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=30.0208&lon=-93.8457&dt=1630213200&appid=2619cb1052a25e553478de5c60d2a13f
http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=30.0208&lon=-93.8457&dt=1630126800&appid=2619cb1052a25e5534

http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=31.0835&lon=-97.6597&dt=1630213200&appid=2619cb1052a25e553478de5c60d2a13f
http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=31.0835&lon=-97.6597&dt=1630126800&appid=2619cb1052a25e553478de5c60d2a13f
http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=31.0835&lon=-97.6597&dt=1630040400&appid=2619cb1052a25e553478de5c60d2a13f
http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=31.0835&lon=-97.6597&dt=1629954000&appid=2619cb1052a25e553478de5c60d2a13f
http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=31.0835&lon=-97.6597&dt=1629867600&appid=2619cb1052a25e553478de5c60d2a13f
------------------------------------------------------------
http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=-31.6187&lon=-60.702&dt=1630213200&appid=2619cb1052a25e553478de5c60d2a13f
http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=-31.6187&lon=-60.702&dt=1630126800&appid=2619cb1052a25

http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=30.5217&lon=-97.8278&dt=1630213200&appid=2619cb1052a25e553478de5c60d2a13f
http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=30.5217&lon=-97.8278&dt=1630126800&appid=2619cb1052a25e553478de5c60d2a13f
http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=30.5217&lon=-97.8278&dt=1630040400&appid=2619cb1052a25e553478de5c60d2a13f
http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=30.5217&lon=-97.8278&dt=1629954000&appid=2619cb1052a25e553478de5c60d2a13f
http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=30.5217&lon=-97.8278&dt=1629867600&appid=2619cb1052a25e553478de5c60d2a13f
------------------------------------------------------------
http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=29.6261&lon=-95.7316&dt=1630213200&appid=2619cb1052a25e553478de5c60d2a13f
http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=29.6261&lon=-95.7316&dt=1630126800&appid=2619cb1052a25